In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import CorrMaster_Sven_v02 as cms
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.markers import MarkerStyle

def plotPC(pcScores, toPlot = (0,1,2)):
    fig = plt.figure(figsize = (16,7))
    ax = fig.gca(projection='3d')
    ax.set_aspect('equal')

    X = pcScores[:,toPlot[0]]
    Y = pcScores[:,toPlot[1]]
    Z = pcScores[:,toPlot[2]]
    
    colors = plt.cm.tab20(range(len(X)))
    #fills = ['bottom','left','top','right','top']
    sizes = [400] * len(X) - np.linspace(1,350,num = len(X))
    for i in range(len(X)):
        ax.scatter(X[i], Y[i], Z[i], color=colors[i%20,:], label = ('MS ' + str(i + 1)), s = sizes[i]) 
                   #marker = MarkerStyle(marker = 'o', fillstyle = fills[i%5]))
    
    # Create cubic bounding box to simulate equal aspect ratio
    max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())
    # Comment or uncomment following both lines to test the fake bounding box:
    #for xb, yb, zb in zip(Xb, Yb, Zb):
    #    ax.plot([xb], [yb], [zb], 'w')
    
    ax.legend(loc = 2)
    
    ax.set_xlabel('PC' + str(toPlot[0] + 1))
    ax.set_ylabel('PC' + str(toPlot[1] + 1))
    ax.set_zlabel('PC' + str(toPlot[2] + 1))
    
    plt.grid()
    
    plt.show()
    
def reshapeAutoCross(x, dim, reshype = 'cross'):
    if reshype is 'cross':
        return np.reshape(x, (dim,dim,dim))
    elif reshype is 'auto':
        ind = ((dim**3) // 2)
        x = np.concatenate((x[:ind],(x[ind],),np.flip(x[:ind], axis = 0)))
        return np.reshape(x, (dim,dim,dim))

    
indAuto = 113491
indCross = 226981

diskMask = np.memmap('ImgData/SandiaIndexed/convM.npy', dtype = np.float32, mode = 'r+', shape = (55,indAuto))
memMask = diskMask[:,:]
del(diskMask)
memMask = np.concatenate((memMask[:,:113490],np.reshape(memMask[:,113490], (55,1)),
                         np.flip(memMask[:,:113490], axis = 0)), axis = 1)


diskConv00 = np.memmap('ImgData/SandiaIndexed/conv00.npy', dtype = np.float32, mode = 'r+', shape = (55,indAuto))
diskConv11 = np.memmap('ImgData/SandiaIndexed/conv11.npy', dtype = np.float32, mode = 'r+', shape = (55,indAuto))
diskConv22 = np.memmap('ImgData/SandiaIndexed/conv22.npy', dtype = np.float32, mode = 'r+', shape = (55,indAuto))
diskConv01 = np.memmap('ImgData/SandiaIndexed/conv01.npy', dtype = np.float32, mode = 'r+', shape = (55,indCross))
diskConv02 = np.memmap('ImgData/SandiaIndexed/conv02.npy', dtype = np.float32, mode = 'r+', shape = (55,indCross))
diskConv12 = np.memmap('ImgData/SandiaIndexed/conv12.npy', dtype = np.float32, mode = 'r+', shape = (55,indCross))

# memConv = diskConv02[:,:]
# memConv = np.concatenate((diskConv00,diskConv11,diskConv22,diskConv01,diskConv02,diskConv12), axis = 1)
memConv = np.concatenate((diskConv00/memMask[:,:indAuto],diskConv11/memMask[:,:indAuto],
                          diskConv22/memMask[:,:indAuto],
                          diskConv01/memMask[:,:],diskConv02/memMask[:,:],diskConv12/memMask[:,:]), axis = 1)

print(memConv.shape)

del(diskConv00)
del(diskConv11)
del(diskConv22)
del(diskConv01)
del(diskConv02)
del(diskConv12)

pca = PCA(n_components=55)
pca.fit(memConv)
X_red = pca.transform(memConv)

In [ ]:
stats00 = reshapeAutoCross(memConv[0,:indCross], 61, reshype = 'cross')
mask = reshapeAutoCross(memMask[0,:], 61, reshype = 'auto')
cms.printStatsInfo(stats00)
cms.plot3DStats(stats00/np.max(stats00), cutoff=30)
cms.plot3DStats(mask/np.max(mask), cutoff=30)
cms.plot3DStats(stats00/mask, cutoff=30)

In [ ]:
from scipy.io import loadmat

checkStat = loadmat('Matlab/stats02.mat')['stats02']

print(checkStat.shape)

checkPy = stats00/mask

print(np.allclose(np.around(checkPy,5),np.around(checkStat,5)))

cms.plot3DStats(checkStat, cutoff=30)
cms.plot3DStats(checkPy, cutoff=30)

In [ ]:
pcEig = 1

PC1eig00 = reshapeAutoCross(pca.components_[pcEig,:indAuto], 61, reshype = 'auto')
PC1eig11 = reshapeAutoCross(pca.components_[pcEig,indAuto:2*indAuto], 61, reshype = 'auto')
PC1eig22 = reshapeAutoCross(pca.components_[pcEig,2*indAuto:3*indAuto], 61, reshype = 'auto')
PC1eig01 = reshapeAutoCross(pca.components_[pcEig,3*indAuto:3*indAuto + indCross], 61, reshype = 'cross')
PC1eig02 = reshapeAutoCross(pca.components_[pcEig,3*indAuto+indCross:3*indAuto+2*indCross], 61, reshype = 'cross')
PC1eig12 = reshapeAutoCross(pca.components_[pcEig,3*indAuto+2*indCross:3*indAuto+3*indCross], 61, reshype = 'cross')

#PC2eig00 = reshapeAutoCross(pca.components_[1,:indAuto], 61, reshype = 'auto')

#PC3eig00 = reshapeAutoCross(pca.components_[2,:indAuto], 61, reshype = 'auto')

cms.plot3DStats(PC1eig00/np.max(abs(PC1eig00)), cutoff=30)
cms.plot3DStats(PC1eig11/np.max(abs(PC1eig11)), cutoff=30)
cms.plot3DStats(PC1eig22/np.max(abs(PC1eig22)), cutoff=30)
cms.plot3DStats(PC1eig01/np.max(abs(PC1eig01)), cutoff=30)
cms.plot3DStats(PC1eig02/np.max(abs(PC1eig02)), cutoff=30)
cms.plot3DStats(PC1eig12/np.max(abs(PC1eig12)), cutoff=30)
cms.plot3DStats(PC2eig00/np.max(abs(PC2eig00)), cutoff=30)
cms.plot3DStats(PC3eig00/np.max(abs(PC3eig00)), cutoff=30)

In [ ]:
plotPC(X_red, toPlot = (0,1,2))
plotPC(X_red, toPlot = (3,4,5))
plotPC(X_red, toPlot = (6,7,8))
plotPC(X_red, toPlot = (9,10,11))

In [ ]:
samples = ['A15', 'A16', 'A17', 'A18', 'A19',
           'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19',
           'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
           'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19',
           'E08', 'E09', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19',
           'F08', 'F09', 'F10', 'F11', 'F12', 'F13', 'F14']

import pandas as pan

pcDF = pan.DataFrame({'Samples': samples})#, 'PC1': X_red[:,0]})

for i in range(X_red.shape[1]):
    name = 'PC' + str(i + 1)
    pcDF[name] = X_red[:,i]
        
propertyDF = pan.read_csv('propertyData.csv')
propertyDF = propertyDF[['Serial_ColRow', '05yield_UnloadingMod_Mpa', 'Modulus_unloading_Gpa',
                         'Strength_ultimate_Mpa', 'Ductility_Percent']]
propertyDF = propertyDF.rename(index = str, columns = {'Serial_ColRow': 'Samples'})

pcPropDF = pan.merge(propertyDF, pcDF, on = 'Samples')
pcPropDF.columns.values

In [ ]:
#prop = 'Ductility_Percent'
prop = '05yield_UnloadingMod_Mpa'
#prop = 'Modulus_unloading_Gpa'

ax = pcPropDF.plot(kind='scatter', x='PC1', y='PC2', c=prop,
                   s=50, figsize = (7,7), colormap = 'hot', sharex = False)
plt.show()

ax = pcPropDF.plot(kind='scatter', x='PC3', y='PC4', c=prop,
                   s=50, figsize = (7,7), colormap = 'hot', sharex = False)
plt.show()

ax = pcPropDF.plot(kind='scatter', x='PC5', y='PC6', c=prop,
                   s=50, figsize = (7,7), colormap = 'hot', sharex = False)
plt.show()

In [ ]:
ax = pcPropDF.corr()[['Ductility_Percent']].plot()
plt.show()

ax = pcPropDF.corr()[['05yield_UnloadingMod_Mpa']].plot()
plt.show()

ax = pcPropDF.corr()[['Modulus_unloading_Gpa']].plot()
plt.show()

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

clm = linear_model.LinearRegression()

#prop = 'Modulus_unloading_Gpa'

oneProp = pcPropDF[[prop, 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
                   'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20',
                   'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28',
                   'PC29', 'PC30']]#, 'PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36',
#                    'PC37', 'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43', 'PC44',
#                    'PC45', 'PC46', 'PC47', 'PC48', 'PC49', 'PC50', 'PC51', 'PC52',
#                    'PC53', 'PC54', 'PC55']]

oneProp = oneProp.sample(frac=1, random_state = 10)


trainSize = 30

train = abs(oneProp.as_matrix()[:trainSize, :])
test = abs(oneProp.as_matrix()[trainSize:, :])

clm.fit(train[:,1:], train[:,0])

In [ ]:
fig = plt.figure(figsize = (12,8))
ax = fig.gca()
ax.scatter(train[:,0], clm.predict(train[:,1:]), color='black', 
            label = 'Train on %d samples'%trainSize)
ax.scatter(test[:,0], clm.predict(test[:,1:]), color='red', 
         label = 'Test on %d samples'%(55-trainSize))

minProp = oneProp[[prop]].min().item()
maxProp = oneProp[[prop]].max().item()

ax.plot(np.linspace(minProp,maxProp), 
         np.linspace(minProp,maxProp), color='blue', linewidth=3)
         
ax.legend(fontsize = 18)

ax.set_xlabel('Actual', fontsize = 20)
ax.set_ylabel('Predicted', fontsize = 20)
ax.set_title(prop, fontsize = 24)
ax.text(s = 'R2 on test = ' + str(round(r2_score(test[:,0],clm.predict(test[:,1:])),2)), 
        y = 0.07, x = 0.77, fontsize = 20,  horizontalalignment='center', verticalalignment='center',
        transform=ax.transAxes)
plt.show()